# Comparing Band-Passes

Targets in CSP DR3 have observations taken at using multiple instruments. This results in multiple physical filters being used for certain band-passes. To minimize the amount of book-keeping we have to do, this notebook justifies why we can't ignore certain bands (even though that would make our lives easier).

#### Table of Contents:

1. <a href='#transmission_filters'>Transmission Filters</a>: Inspect the CSP transmission filters and the number of observations in each band.
1. <a href='#picking_y_band'>Picking a "Y" Band</a>: Comparing the `Y` and `Ydw` bands.


In [ ]:
import numpy as np
import sys

import sncosmo
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
from sndata.csp import dr3

sys.path.insert(0, '../')
from analysis import utils

dr3.register_filters(force=True)


## Transmission Filters <a id='transmission_filters'></a>

We review the CSP filter transmission curves and take note that there are multiple "duplicate" filters. This is due to observations being performed on multiple telescopes or, in the case of V band, a filter being broken and subsequently replaced.

In [ ]:
plt.figure(figsize=(10, 5))
for band_name in dr3.band_names:
    band = sncosmo.get_bandpass(band_name)
    band_label = band_name.split('_')[-1]
    
    plt.plot(band.wave, band.trans, label=band_label)
    plt.xlabel('Wavelength (A)')
    plt.ylabel('Transmission')

plt.legend()  
plt.ylim(0, 1)
plt.tight_layout()


In the following figure, we note that observations are not available in all bands. Not only is the number of observations in the NIR is significantly less than in the optical, the only bands used observed with the Dupont telescope in the NIR is the `Ydw` band.

In [ ]:
data_count = {b: [] for b in dr3.band_names}
for table in dr3.iter_data(format_sncosmo=True):
    for band in dr3.band_names:
        data_count[band].append(len(table[table['band'] == band]))


In [ ]:
plot_order = 'ugriBVYJH'
fig, axes = plt.subplots(4, 3, figsize=(20, 12))
for band, axis in zip(plot_order, axes.flatten()):
    axis.hist(data_count['csp_dr3_' + band], label=band)
    axis.set_xlabel('Number of Observations')
    
axes[1, 2].hist(data_count['csp_dr3_V0'], label='V0')
axes[1, 2].hist(data_count['csp_dr3_V1'], label='V01')

axes[3, 0].hist(data_count['csp_dr3_Ydw'], label='Ydw')
axes[3, 1].hist(data_count['csp_dr3_Jdw'], label='Jdw')
axes[3, 2].hist(data_count['csp_dr3_Hdw'], label='Hdw')

[axis.legend() for axis in axes.flatten()]
plt.tight_layout()
plt.show()


## Picking a "Y" Band <a id='picking_y_band'></a>

Unfortunately, we can't just pick use observations from a single `Y` band since some targets may have a significant number of observations in both bands.

In [ ]:
plt.scatter(data_count['csp_dr3_Y'], data_count['csp_dr3_Ydw'])
plt.xlabel('Number of Y Observations')
plt.ylabel('Number of Ydw Observations')

We also can't justify ignoring one of the `Y` bands based on its phase coverage since observations can cover a significant portion of the light curve. (Note that the below figure only includes objects with a published `t0` value)

In [ ]:
phase_limits = dict()
table_iterator = dr3.iter_data(format_sncosmo=True, filter_func=utils.filter_has_csp_data)
for table in table_iterator:
    t0 = utils.get_csp_t0(table.meta['obj_id'])
    for band in ['csp_dr3_Y', 'csp_dr3_Ydw']:
        band_data = table[table['band'] == band]
        if band_data:
            time = band_data['time'] - t0
            phase_limits.setdefault(band, [])
            phase_limits[band].append((min(time), max(time)))
    

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
for band, axis in zip(phase_limits, axes.flatten()):
    hist_data = np.transpose(phase_limits[band])

    axis.hist(hist_data[1], label=f'Max phase')
    axis.hist(hist_data[0], label=f'Min phase')

    av_min = np.average(hist_data[0])
    av_max = np.average(hist_data[1])
    axis.axvline(av_min, linestyle='-', color='k', label=f'Avg min = {av_min:.2}')
    axis.axvline(av_max, linestyle='--', color='k', label=f'Avg min = {av_max:.2}')

    axis.set_title(band.split('_')[-1])
    axis.set_xlim(-20, 150)
    axis.xaxis.set_minor_locator(MultipleLocator(10))              
    axis.legend()
    
plt.tight_layout()

